In [ ]:
import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

import coffea

from coffea.nanoevents import NanoEventsFactory

import itertools
from itertools import combinations

In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")


In [ ]:
data_dir = '/home/bellis/top_data/NANOAOD/'

# Laptop
#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
#infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

# Beth Harmon
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'
#infilename = 'TTToSemiLeptonic_UL_2018.root' # Also desktop home computer
#infilename = 'Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer

# Desktop home computer
infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'



infile = uproot.open(data_dir + infilename)


In [ ]:
################################################################################
# Generate the indices for the diferent combinations
################################################################################
def generate_event_topology_indices(njets,nleps,verbose=False):

    index_combinations = []

    if njets<5 or nleps<1:
        return [[None,None,None,None,None,None]]

    jetindices = np.arange(njets,dtype=int)
    lepindices = np.arange(nleps,dtype=int)

    x = combinations(jetindices,3)

    for had in x:
        remaining = np.delete(jetindices, np.argwhere( (jetindices==had[0]) | (jetindices==had[1]) | (jetindices==had[2]) ))
        bnv = combinations(remaining,2)
        for b in bnv:
            for lep in lepindices:
                index_combinations.append([had[0],had[1],had[2],b[0],b[1],lep])

                if verbose:
                    print(had,b,lep)

    return np.array(index_combinations)


In [ ]:
#-------------------------------#

njet = infile["Events"]["nJet"].array()#[0:1000]
nmuon = infile["Events"]["nMuon"].array()#[0:1000]

min_njets = 5
max_njets = 8

mask_njets = (njet>=min_njets) & (njet<=max_njets)
#-------------------------------#

muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")#[0:1000]
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:1000]

muons = ak.zip({
    "pt": muon_branch_arrays[mask_njets]["Muon_pt"],
    "phi": muon_branch_arrays[mask_njets]["Muon_phi"],
    "eta": muon_branch_arrays[mask_njets]["Muon_eta"],
    "mass": muon_branch_arrays[mask_njets]["Muon_mass"],
    "charge": muon_branch_arrays[mask_njets]["Muon_charge"],
}, with_name="Momentum4D")

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btag": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

jet_combos = ak.combinations(jets, 5)
#muon_combos = ak.combinations(muons, 1)
muon_combos = ak.zip((muons,))

jet1, jet2, jet3, jet4, jet5 = ak.unzip(jet_combos)
muon1 = ak.unzip(muon_combos)

### Need this for later

njet_masked = infile["Events"]["nJet"].array()[mask_njets]#[0:1000]
nmuon_masked = infile["Events"]["nMuon"].array()[mask_njets]#[0:1000]

In [ ]:
print(len(njet), len(njet_masked))

In [ ]:
# Generate the combinations

combos = []
event_indices = []

icount = 0
for i,(nj,nm) in enumerate(zip(njet_masked,nmuon_masked)):
    
    combo = generate_event_topology_indices(njets=nj,nleps=nm)

    if combo[0][0] is None:
        continue
    
    for c in combo:
        combos.append(np.array(c))
    event_indices += (i*np.ones(len(combo),dtype=int)).tolist()
    icount += 1

combos = np.array(combos)

print(combos.shape)

In [ ]:
jet1 = jets[(event_indices,combos.transpose()[0])]
jet2 = jets[(event_indices,combos.transpose()[1])]
jet3 = jets[(event_indices,combos.transpose()[2])]
jet4 = jets[(event_indices,combos.transpose()[3])]
jet5 = jets[(event_indices,combos.transpose()[4])]

muon = muons[(event_indices,combos.transpose()[5])]

In [ ]:
def top_variables(jets, decay_type='had'):    
    
    # btag
    var1 = jets[0].btag
    var2 = jets[1].btag
    var3 = None
    if decay_type == 'had':
        var3 = jets[2].btag
    elif decay_type == 'bnv':
        var3 = jets[2].charge
        
    # Boost to CM of 3-jet (or 2-jet + lepton) system
    top_p4 = jets[0] + jets[1] + jets[2]
    boost_p4 = top_p4
    
    boost_p4 = ak.with_field(boost_p4, 173.0, "tau")

    jet1_boosted = jets[0].boostCM_of(boost_p4)
    jet2_boosted = jets[1].boostCM_of(boost_p4)
    jet3_boosted = jets[2].boostCM_of(boost_p4)

    # Need to do this so we only sort the 2 jets for bnv decays
    if decay_type=='had':

        # Sort things by the magnitude of momentum in the top-CM frame    
        sort_by = np.array([jet1_boosted.mag, jet2_boosted.mag, jet3_boosted.mag]).transpose()
        idx = np.argsort(sort_by)

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = sort_by
        jet3_boosted_mag,jet2_boosted_mag,jet1_boosted_mag = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jets[0], jets[1], jets[2]]).transpose()
        jet3,jet2,jet1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()


        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jet1_boosted, jet2_boosted, jet3_boosted]).transpose()
        jet3_boosted,jet2_boosted,jet1_boosted = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([var1, var2, var3]).transpose()
        var3, var2, var1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Convert back to awkward
        jet1 = ak.Array(jet1,with_name="Momentum4D")
        jet2 = ak.Array(jet2,with_name="Momentum4D")
        jet3 = ak.Array(jet3,with_name="Momentum4D")

        jet1_boosted = ak.Array(jet1_boosted,with_name="Momentum4D")
        jet2_boosted = ak.Array(jet2_boosted,with_name="Momentum4D")
        jet3_boosted = ak.Array(jet3_boosted,with_name="Momentum4D")

        '''
        #jet1_pt,jet2_pt,jet3_pt = jet1.pt,jet2.pt,jet3.pt
        print(type(jet1))
        print(jet1.fields)
        jet1_pt = jet1.rho # This is pt
        jet2_pt = None
        jet3_pt = None
        #array_to_sort = np.array([jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt]).transpose()
        jet1_boosted_pt,jet2_boosted_pt,jet3_boosted_pt = jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt#np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        had_dR12 = jet1.deltaR(jet2)
        had_dR13 = jet1.deltaR(jet3)
        had_dR23 = jet2.deltaR(jet3)
        had_dR1_23 = jet1.deltaR(jet2 + jet3)

        had_p12 = jet1 + jet2
        had_p13 = jet1 + jet3
        had_p23 = jet2 + jet3

        had_dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
        had_dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
        had_dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
        had_dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
        '''
        
    elif decay_type=='bnv':
   
        # Sort things by the magnitude of momentum in the top-CM frame    
        sort_by = np.array([jet1_boosted.mag, jet2_boosted.mag]).transpose()
        idx = np.argsort(sort_by)

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = sort_by
        jet2_boosted_mag,jet1_boosted_mag = np.take_along_axis(array_to_sort, idx, axis=1).transpose()
        jet3_boosted_mag = jet3_boosted.mag

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jets[0], jets[1]]).transpose()
        jet2,jet1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()
        jet3 = jets[2]

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([jet1_boosted, jet2_boosted]).transpose()
        jet2_boosted,jet1_boosted = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Sort these "backward" because we want the highest momentum first
        array_to_sort = np.array([var1, var2]).transpose()
        var2, var1 = np.take_along_axis(array_to_sort, idx, axis=1).transpose()

        # Convert back to awkward
        jet1 = ak.Array(jet1,with_name="Momentum4D")
        jet2 = ak.Array(jet2,with_name="Momentum4D")
        jet3 = ak.Array(jet3,with_name="Momentum4D")

        jet1_boosted = ak.Array(jet1_boosted,with_name="Momentum4D")
        jet2_boosted = ak.Array(jet2_boosted,with_name="Momentum4D")
        jet3_boosted = ak.Array(jet3_boosted,with_name="Momentum4D")

    #jet1_pt,jet2_pt,jet3_pt = jet1.pt,jet2.pt,jet3.pt
    print(type(jet1))
    print(jet1.fields)
    jet1_pt = jet1.rho # This is pt
    jet2_pt = jet1.rho
    jet3_pt = jet1.rho
    #array_to_sort = np.array([jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt]).transpose()
    jet1_boosted_pt,jet2_boosted_pt,jet3_boosted_pt = jet1_boosted.pt, jet2_boosted.pt,jet3_boosted.pt#np.take_along_axis(array_to_sort, idx, axis=1).transpose()

    dR12 = jet1.deltaR(jet2)
    dR13 = jet1.deltaR(jet3)
    dR23 = jet2.deltaR(jet3)
    dR1_23 = jet1.deltaR(jet2 + jet3)
    dR3_12 = jet3.deltaR(jet1 + jet2)


    p12 = jet1 + jet2
    p13 = jet1 + jet3
    p23 = jet2 + jet3

    dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
    dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
    dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
    dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
    dThetaCM3_12 = jet3_boosted.deltaangle(jet1_boosted + jet2_boosted)

    #xsorted,ysorted,zsorted = xsort.transpose()
    
    return top_p4.mass, p12.mass, p13.mass, p23.mass, \
           dR12, dR13, dR23, dR1_23, dR3_12,\
           dThetaCM12, dThetaCM13, dThetaCM23, dThetaCM1_23, dThetaCM3_12, \
           jet1_boosted_pt, jet2_boosted_pt, jet3_boosted_pt, \
           jet1_boosted_mag, jet2_boosted_mag, jet3_boosted_mag, \
           var1, var2, var3


def event_hypothesis(jets, lepton):
    had_p4 = jets[0] + jets[1] + jets[2]
    bnv_p4 = jets[3] + jets[4] + lepton
    
    angle = had_p4.deltaangle(bnv_p4)

    had_variables = top_variables([jets[0], jets[1], jets[2]], decay_type='had')
    print("here")
    print(type(jets[3]))
    bnv_variables = top_variables([jets[3], jets[4], lepton], decay_type='bnv')
    
    return had_variables, bnv_variables, angle



In [ ]:
had_variables, bnv_variables, angle = event_hypothesis([jet1, jet2, jet3, jet4, jet5], muon)


In [ ]:
#values[2]

In [ ]:
for values in [had_variables, bnv_variables]:
    plt.figure(figsize=(16,12))

    for i in range(len(values)):
        #x = ak.flatten(values[i]).to_numpy()   
        #print(type(values[i]))
        x = values[i]
        if type(x) == ak.highlevel.Array:
            x = values[i].to_numpy()
        #print(type(x))

        #print(len(x),x)
        #print(len(x[x==x]))
        x[x==-np.inf] = -999
        x[x==np.inf] = -999
        plt.subplot(5,5,i+1)
        if i>=15 and i<=17:
            plt.hist(x[x==x],bins=100,range=(0,250))
        else:
            plt.hist(x[x==x],bins=100)

plt.figure()
plt.hist(np.cos(angle),bins=100);

In [ ]:
plt.hist(np.unique(had_variables[-3]),bins=100,range=(-3,3));#, range=(0,1200));

In [ ]:
plt.hist(had_variables[-6],bins=100,range=(0,250));
plt.hist(had_variables[-5],bins=100,range=(0,250),alpha=0.6);
plt.hist(had_variables[-4],bins=100,range=(0,250),alpha=0.4);

In [ ]:
plt.hist(had_variables[-9],bins=100,range=(0,150));
plt.hist(had_variables[-8],bins=100,range=(0,150),alpha=0.6);
plt.hist(had_variables[-7],bins=100,range=(0,150),alpha=0.4);

In [ ]:
x = values[0]

In [ ]:
x.to_numpy()

In [ ]:
x = ak.Array([12, 14, 19, 16]).to_numpy()
y = ak.Array([10, 15, 18, 13]).to_numpy()

In [ ]:
z = np.array([x,y])

In [ ]:
zt = z.transpose()

In [ ]:
idx = np.argsort(zt).transpose()

In [ ]:
idx

In [ ]:
x[idx[0]==0]